In [13]:
from theano.sandbox import cuda
cuda.use('gpu2')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

ERROR (theano.sandbox.cuda): nvcc compiler not found on $PATH. Check your nvcc installation and try again.


In [14]:
%matplotlib inline

In [15]:
import utils; reload(utils)
from utils import * 
from __future__ import division, print_function 

In [60]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Setup
cell to markdown: 'm', cell to code: 'y'

In [61]:
batch_size=64

In [62]:
from keras.datasets import mnist

In [63]:
(X_train, y_train) , (X_test, y_test) = mnist.load_data()

In [66]:
X_train.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [64]:
X_train=X_train[:10000]
y_train=y_train[:10000]

In [65]:
# shape of the input space ==> 28 * 28 * 1 (grayscale)
(X_train.shape, y_train.shape)

((10000, 28, 28), (10000,))

In [22]:
(X_test.shape, y_train.shape)

((10000, 28, 28), (10000,))

Fix the data shape - it has one channel (grayscale) and we need to specify this explicitly

In [23]:
X_train = np.expand_dims(X_train, 1)
X_test = np.expand_dims(X_test, 1)

In [24]:
X_train.shape

(10000, 1, 28, 28)

What do the labels look like?

In [25]:
y_train[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

Use onehot encoding for the labels

In [26]:
onehot(y_train)[:10]

array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]])

In [27]:
y_train = onehot(y_train)
y_test = onehot(y_test)

# Linear model

In [28]:
def dummy(x):
    return x

In [29]:
def get_linear_model(): 
    model = Sequential([
        # The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.
        Lambda(dummy, input_shape=(1, 28, 28)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    # Adam is an optimizer: https://keras.io/optimizers/#adam
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [30]:
lm = get_linear_model()

/Users/roland/anaconda/envs/fastai/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/Users/roland/anaconda/envs/fastai/lib/python2.7/site-packages/keras/engine/topology.py:1811: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "sequential_1_model" was not an Input tensor, it was generated by layer lambda_1.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: lambda_input_1
  str(x.name))


In [31]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 1, 28, 28)     0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
flatten_1 (Flatten)              (None, 784)           0           lambda_1[0][0]                   
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 10)            7850        flatten_1[0][0]                  
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
____________________________________________________________________________________________________


### Start by training fast for 1 iteration, then slow down


In [32]:
lm.optimizer.lr=0.1
lm.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test), nb_epoch=5)

Train on 10000 samples, validate on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 0s - loss: 14.1676 - acc: 0.1208 - val_loss: 14.1638 - val_acc: 0.1212
Epoch 2/5
10000/10000 [==============================] - 0s - loss: 14.1005 - acc: 0.1251 - val_loss: 13.9144 - val_acc: 0.1366
Epoch 3/5
10000/10000 [==============================] - 0s - loss: 13.7618 - acc: 0.1461 - val_loss: 13.8712 - val_acc: 0.1393
Epoch 4/5
10000/10000 [==============================] - 0s - loss: 13.8567 - acc: 0.1403 - val_loss: 13.8714 - val_acc: 0.1393
Epoch 5/5
10000/10000 [==============================] - 0s - loss: 13.8567 - acc: 0.1403 - val_loss: 13.8714 - val_acc: 0.1393


In [33]:
lm.optimizer.lr=0.01
lm.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test), nb_epoch=5)

Train on 10000 samples, validate on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 0s - loss: 13.8567 - acc: 0.1403 - val_loss: 13.8714 - val_acc: 0.1393
Epoch 2/5
10000/10000 [==============================] - 0s - loss: 13.8567 - acc: 0.1403 - val_loss: 13.8714 - val_acc: 0.1393
Epoch 3/5
10000/10000 [==============================] - 0s - loss: 13.8567 - acc: 0.1403 - val_loss: 13.8714 - val_acc: 0.1393
Epoch 4/5
10000/10000 [==============================] - 0s - loss: 13.8567 - acc: 0.1403 - val_loss: 13.8714 - val_acc: 0.1393
Epoch 5/5
10000/10000 [==============================] - 0s - loss: 13.8567 - acc: 0.1403 - val_loss: 13.8714 - val_acc: 0.1393


## Met de imagegen

In [34]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [35]:
lm_img_gen = get_linear_model()

/Users/roland/anaconda/envs/fastai/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_2 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
/Users/roland/anaconda/envs/fastai/lib/python2.7/site-packages/keras/engine/topology.py:1811: UserWarning: Model inputs must come from a Keras Input layer, they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to "sequential_2_model" was not an Input tensor, it was generated by layer lambda_2.
Note that input tensors are instantiated via `tensor = Input(shape)`.
The tensor that caused the issue was: lambda_input_2
  str(x.name))


In [36]:
lm_img_gen.optimizer.lr=0.1
lm_img_gen.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
10000/10000 [==============================] - 2s - loss: 12.7013 - acc: 0.2117 - val_loss: 12.7818 - val_acc: 0.2069


Deze performance is waardeloos...

## Use normalization

In [37]:
mean_pixel = X_train.mean().astype(np.float32)
std_pixel = X_train.std().astype(np.float32)

In [38]:
def normalize_input(x):
    return ((x - mean_pixel) / std_pixel)

In [39]:
def get_linair_model_normalized(): 
    model = Sequential([
        # The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.
        Lambda(normalize_input, input_shape=(1, 28, 28)),
        Flatten(),
        Dense(10, activation='softmax')
    ])
    # Adam is an optimizer: https://keras.io/optimizers/#adam
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [40]:
lmnorm = get_linair_model_normalized()

/Users/roland/anaconda/envs/fastai/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_3 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [41]:
lmnorm.optimizer.lr=0.1
lmnorm.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test), nb_epoch=1)

Train on 10000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 0s - loss: 11.3982 - acc: 0.2850 - val_loss: 11.4692 - val_acc: 0.2867


In [42]:
lmnorm.optimizer.lr=0.01
lmnorm.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test), nb_epoch=5)

Train on 10000 samples, validate on 10000 samples
Epoch 1/5
10000/10000 [==============================] - 0s - loss: 11.3981 - acc: 0.2918 - val_loss: 11.4151 - val_acc: 0.2912
Epoch 2/5
10000/10000 [==============================] - 0s - loss: 11.3602 - acc: 0.2941 - val_loss: 11.4190 - val_acc: 0.2910
Epoch 3/5
10000/10000 [==============================] - 0s - loss: 11.3637 - acc: 0.2944 - val_loss: 11.5009 - val_acc: 0.2854
Epoch 4/5
10000/10000 [==============================] - 0s - loss: 11.3622 - acc: 0.2945 - val_loss: 11.5165 - val_acc: 0.2847
Epoch 5/5
10000/10000 [==============================] - 0s - loss: 11.3619 - acc: 0.2942 - val_loss: 11.3823 - val_acc: 0.2933


# Use the image generator

In [43]:
gen = image.ImageDataGenerator()
batches = gen.flow(X_train, y_train, batch_size=64)
test_batches = gen.flow(X_test, y_test, batch_size=64)

In [44]:
lm3 = get_linair_model_normalized()

/Users/roland/anaconda/envs/fastai/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_4 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


Op de een of andere manier verbetert de image generator de performance dramatisch, maar ik snap ff niet waarom

In [45]:
lm3.optimizer.lr=0.1
lm3.fit_generator(batches, batches.n, nb_epoch=1, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
10000/10000 [==============================] - 2s - loss: 8.1672 - acc: 0.4806 - val_loss: 8.2354 - val_acc: 0.4854


In [46]:
lm3.optimizer.lr=0.01
lm3.fit_generator(batches, batches.n, nb_epoch=5, 
                    validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/5
10000/10000 [==============================] - 2s - loss: 8.1172 - acc: 0.4936 - val_loss: 8.2367 - val_acc: 0.4866
Epoch 2/5
10000/10000 [==============================] - 2s - loss: 8.0634 - acc: 0.4978 - val_loss: 8.1732 - val_acc: 0.4912
Epoch 3/5
10000/10000 [==============================] - 2s - loss: 8.0022 - acc: 0.5012 - val_loss: 8.2052 - val_acc: 0.4890
Epoch 4/5
10000/10000 [==============================] - 2s - loss: 7.9795 - acc: 0.5033 - val_loss: 8.2258 - val_acc: 0.4877
Epoch 5/5
10000/10000 [==============================] - 2s - loss: 8.0738 - acc: 0.4974 - val_loss: 8.1678 - val_acc: 0.4921


# Fully connected model (has dense layer)

In [47]:
def get_fc_model(): 
    model = Sequential([
        # The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.
        Lambda(normalize_input, input_shape=(1, 28, 28)),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')
    ])
    # Adam is an optimizer: https://keras.io/optimizers/#adam
    # This is a multiclass problem, so we use categorical_crossentropy rather than binary_crossentropy
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [48]:
fc_model = get_fc_model()

/Users/roland/anaconda/envs/fastai/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_5 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [49]:
fc_model.fit_generator(batches, batches.n, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
10000/10000 [==============================] - 3s - loss: 2.1130 - acc: 0.7320 - val_loss: 1.9651 - val_acc: 0.8312


In [50]:
fc_model.optimizer.lr=0.1
fc_model.fit_generator(batches, batches.n, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
10000/10000 [==============================] - 3s - loss: 1.8315 - acc: 0.8644 - val_loss: 1.7091 - val_acc: 0.8799


In [52]:
fc_model.optimizer.lr=0.01
fc_model.fit_generator(batches, batches.n, nb_epoch=5, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/5
10000/10000 [==============================] - 3s - loss: 0.8024 - acc: 0.9427 - val_loss: 0.8075 - val_acc: 0.9146
Epoch 2/5
10000/10000 [==============================] - 3s - loss: 0.7029 - acc: 0.9495 - val_loss: 0.7281 - val_acc: 0.9176
Epoch 3/5
10000/10000 [==============================] - 3s - loss: 0.6243 - acc: 0.9523 - val_loss: 0.6621 - val_acc: 0.9184
Epoch 4/5
10000/10000 [==============================] - 3s - loss: 0.5598 - acc: 0.9527 - val_loss: 0.6089 - val_acc: 0.9171
Epoch 5/5
10000/10000 [==============================] - 3s - loss: 0.4996 - acc: 0.9573 - val_loss: 0.5618 - val_acc: 0.9195


# Use convolutions

In [53]:
def get_cnn_model(): 
    model = Sequential([
        # The first layer in a Sequential model must get an `input_shape` or `batch_input_shape` argument.
        Lambda(normalize_input, input_shape=(1, 28, 28)),
        Convolution2D(32,3,3, activation='relu'),
        Convolution2D(32,3,3, activation='relu'),
        MaxPooling2D(),
#         Convolution2D(64,3,3, activation='relu'),
#         Convolution2D(64,3,3, activation='relu'),
#         MaxPooling2D(),
        Flatten(),
        Dense(512, activation='softmax'),
        Dense(10, activation='softmax')
    ])
    # Adam is an optimizer: https://keras.io/optimizers/#adam
    # This is a multiclass problem, so we use categorical_crossentropy rather than binary_crossentropy
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [54]:
cnn_model = get_cnn_model()

/Users/roland/anaconda/envs/fastai/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_6 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 1, 28, 28)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [56]:
cnn_model.fit_generator(batches, batches.n, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
10000/10000 [==============================] - 63s - loss: 1.8755 - acc: 0.8008 - val_loss: 1.7593 - val_acc: 0.8164


In [57]:
cnn_model.optimizer.lr=0.1
cnn_model.fit_generator(batches, batches.n, nb_epoch=1, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/1
10000/10000 [==============================] - 45s - loss: 1.6532 - acc: 0.8373 - val_loss: 1.5406 - val_acc: 0.8583


In [59]:
cnn_model.optimizer.lr=0.01
cnn_model.fit_generator(batches, batches.n, nb_epoch=5, validation_data=test_batches, nb_val_samples=test_batches.n)

Epoch 1/5
10000/10000 [==============================] - 27s - loss: 1.2482 - acc: 0.8779 - val_loss: 1.1812 - val_acc: 0.8665
Epoch 2/5
10000/10000 [==============================] - 28s - loss: 1.0891 - acc: 0.8785 - val_loss: 1.0279 - val_acc: 0.8736
Epoch 3/5
10000/10000 [==============================] - 29s - loss: 0.9469 - acc: 0.8821 - val_loss: 0.9047 - val_acc: 0.8741
Epoch 4/5
10000/10000 [==============================] - 29s - loss: 0.8272 - acc: 0.8884 - val_loss: 0.8011 - val_acc: 0.8761
Epoch 5/5
10000/10000 [==============================] - 29s - loss: 0.7368 - acc: 0.8842 - val_loss: 0.7211 - val_acc: 0.8777


# https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/ 
The training dataset is structured as a 3-dimensional array of instance, image width and image height. For a multi-layer perceptron model we must reduce the images down into a vector of pixels. In this case the 28×28 sized images will be 784 pixel input values.

We can do this transform easily using the reshape() function on the NumPy array. We can also reduce our memory requirements by forcing the precision of the pixel values to be 32 bit, the default precision used by Keras anyway.

In [182]:
# flatten 28*28 images to a 784 vector for each image
(X_train2, y_train2), (X_test2, y_test2) = mnist.load_data()
num_pixels = X_train2.shape[1] * X_train2.shape[2]
X_train_flat = X_train2.reshape(X_train2.shape[0], num_pixels).astype('float32')
X_test_flat = X_test2.reshape(X_test2.shape[0], num_pixels).astype('float32')

In [183]:
# normalize inputs from 0-255 to 0-1
X_train2 = X_train2 / 255
X_test2 = X_test2 / 255

In [184]:
# one hot encode outputs
y_train2 = np_utils.to_categorical(y_train2)
y_test2 = np_utils.to_categorical(y_test2)
num_classes = y_test.shape[1]

The model is a simple neural network with one hidden layer with the same number of neurons as there are inputs (784). A rectifier activation function is used for the neurons in the hidden layer.

A softmax activation function is used on the output layer to turn the outputs into probability-like values and allow one class of the 10 to be selected as the model’s output prediction. Logarithmic loss is used as the loss function (called categorical_crossentropy in Keras) and the efficient ADAM gradient descent algorithm is used to learn the weights.

We can now fit and evaluate the model. The model is fit over 10 epochs with updates every 200 images. The test data is used as the validation dataset, allowing you to see the skill of the model as it trains. A verbose value of 2 is used to reduce the output to one line for each training epoch.

In [185]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(num_pixels, input_dim=num_pixels, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [186]:
# build the model
model = baseline_model()
# Fit the model
model.fit(X_train2, y_train2, validation_data=(X_test2, y_test2), nb_epoch=10, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test2, y_test2, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

ValueError: Error when checking model input: expected dense_input_4 to have 2 dimensions, but got array with shape (60000, 28, 28)